# 20 km WRF restacking pipeline

This pipeline restructures the raw 20km WRF outputs that cover Alaska and the surrounding regions (created by P Bieniek) into more user-friendly files that can be easily imported into popular GIS software. This WRF dataset consists of hourly and daily outputs for one reanalysis, ERA-Interim, and two GCMs, GFDL-CM3, and NCAR-CCSM4. This pipeline is designed to be executed entirely from this notebook.

This is a rather complicated SNAP data pipeline. It works on a large amount of data (~300 GB for a single model / scenario / year, so that's over 90 TB for $2 * 95 + 2 * 35 + 35$ model / scenario / year combinations), creates a large number of final data files (>10k), and makes use of slurm, specific directory structure / file management, and asyncronous execution ability (i.e. re-run certain steps, run steps for only certain variables, etc).

### Processing strategy

The default configuration for this pipeline is to process all available data - all year / variable / model / scenario combinations possible. However, a supplemental "interactive" notebook is provided as well for more fine-grained control, being set up to restack a single year's worth of data for a single variable / model / scenario combination. This is the lowest level of control offered. Find that notebook here - [restack_20km_interactive.ipynb](restack_20km_interactive.ipynb).

As seen below, the input data are grouped by model and scenario names and are consistently structured - hourly and daily WRF model outputs grouped by yearly folders.

Given the volume of this dataset, restacking all of the data in a single sequence does not make sense. Thus, execution of this notebook is broken up into what will be referred to as "WRF groups", which is simply the model + scenario combination being processed. This is the natural grouping given the way the dataset is organized on the current filesystem.

## Pipeline

The pipeline starts here, and is implemented for a single WRF group - the historical data of the NCAR-CCSM4 model - with descriptions of what is being done and why. Processing the remaining 4 WRF groups is done in an abbreviated fashion using the same code in this first section. Use this table of contents to jump to the section you wish to process:


* [NCAR-CCSM4 Historical](#NCAR-CCSM4-Historical)
* [GFDL-CM3 Historical](#GFDL-CM3-Historical)
* [NCAR-CCSM4 RCP 8.5](#NCAR-CCSM4-RCP-8.5)
* [GFDL-CM3 RCP 8.5](#GFDL-CM3-RCP-8.5)
* [ERA-Interim](#ERA-Interim)

First, we provide some background info on the file structure and pipeline.

    The WRF outputs of interest from different runs of model / scenario may be in separate directories, but there is consistency in file structure across all groups - all `hourly` and `daily` directories have annual subgroups consisting of the WRF outputs to be restacked:

In [13]:
ls /archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly | head -5

1970/
1971/
1972/
1973/
1974/


In [14]:
ls /archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly/ | tail -6

2003/
2004/
2005/
nohup.out
orgdata.sh*


In [18]:
ls /archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly/1979 | head -5

dailylog.out*
WRFDS_d01.1979-01-01_00.nc*
WRFDS_d01.1979-01-01_01.nc*
WRFDS_d01.1979-01-01_02.nc*
WRFDS_d01.1979-01-01_03.nc*
ls: write error


This structure applies for all outputs, and exists for the following model / scenario / year combinations:

* ERA-Interim
    * "historical": 1979-2015
* GFDL-CM3
    * historical: 1970-2006
    * RCP 8.5: 2006-2100
* NCAR-CCSM4
    * historical: 1970-2005
    * RCP 8.5: 2005-2100

#### System

This pipeline is being developed on the Chinook cluster:

In [1]:
!uname -a

Linux chinook00.rcs.alaska.edu 2.6.32-754.35.1.el6.61015g0000.x86_64 #1 SMP Mon Dec 21 12:41:07 EST 2020 x86_64 x86_64 x86_64 GNU/Linux


This pipeline makes use of slurm and multiple cores / compute nodes for processing in reasonable time.

In [5]:
!sinfo -V

slurm 19.05.7


#### Execution

This notebook should be executed on a per-section basis to process the entire dataset. In other words, don't expect to run this entire notebook at once! The preferred workflow should be to complete a single WRF group-based section, commit changes which serves to document the processing done, and move on to the next section.

To process only subsets of the target dataset, which might be done for fixing an issue or re-processing some failed runs, use the [interactive restack_20km notebook](restack_20km_interactive.ipynb).

#### Environment

Instead of relying on environment variables, this pipeline utilizes user-supplied parameters specified in the cells of this notebook by simply assigning values to variables prior to executing any processing code cells.

The following variables are used throughout each singular run of the pipeline (i.e., each section of this notebook) and so will be set in each section:

* `base_dir` - Full path to the directory that will contain all ancillary and intermediate files that will be kept, such as scripts for slurm / `sbatch`
* `output_dir` - Full path to the directory that will contain the final output data (will be the same as `base_dir` here but specified separately for consistency with other SNAP pipelines)
* `scratch_dir` - Full path to the scratch directory that raw WRF outputs will be copied to prior to processing them
    * This pipelines works with WRF outputs that are on a mounted file system, and so can be copied over to scratch space and removed when done to improve IO and avoid the need to keep them in the `base_dir`.
* `group`: Encoded value specifying the WRF group being worked on, which is just a combination of the model and scenario (or just model, in terms of ERA-Interim).  One of [`era_interim`, `gfdl_hist`, `ccsm_hist`, `gfdl_rcp85`, `ccsm_rcp85`]. The correct directory to look in is then imported from the `luts.py` file.
* `slurm_email` - String containing email address to use for failed slurm notifications
* `conda_init_script` - This is currently specific to Chinook. This is the path to a script that contains commands for initializing the shells on the compute nodes to use `conda activate`, has the typical commands seen in `~/.bashrc` after installing conda:

In [19]:
cat ~/init_conda.sh

#!/bin/bash

# >>> conda initialize >>>
# !! Contents within this block are managed by 'conda init' !!
__conda_setup="$('/home/kmredilla/miniconda3/bin/conda' 'shell.bash' 'hook' 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__conda_setup"
else
    if [ -f "/home/kmredilla/miniconda3/etc/profile.d/conda.sh" ]; then
        . "/home/kmredilla/miniconda3/etc/profile.d/conda.sh"
    else
        export PATH="/home/kmredilla/miniconda3/bin:$PATH"
    fi
fi
unset __conda_setup
# <<< conda initialize <<< 



With that, we are ready to run the pipeline for a single WRF group.

# NCAR-CCSM4 Historical

## 0 - Setup

Set the "user" parameters - things that will likely change between different SNAP users running this pipeline:

In [1]:
# scratch space where data will be copied for performant reading / writing
scratch_dir = "/center1/DYNDOWN/kmredilla/wrf_data"
slurm_email = "kmredilla@alaska.edu"
conda_init_script = "/home/kmredilla/init_conda.sh"

Set up imports and remaining relevant file and directory paths:

In [2]:
import os
import time
from multiprocessing import Pool
from pathlib import Path
import tqdm
import numpy as np
import pandas as pd
import xarray as xr
# codebase
import luts
import restack_20km as main


base_dir = Path("/import/SNAP/wrf_data/project_data/wrf_data")
anc_dir = base_dir.joinpath("ancillary")
anc_dir.mkdir(exist_ok=True)
# monthly WRF file to serve as template
template_fp = anc_dir.joinpath("monthly_PCPT-gfdlh.nc")
# WRF geogrid file for correctly projecting data and rotating wind data
geogrid_fp = anc_dir.joinpath("geo_em.d01.nc")
# final output directory for data
output_dir = Path("/import/SNAP/wrf_data/project_data/wrf_data/restacked")
output_dir.mkdir(exist_ok=True)
scratch_dir = Path(scratch_dir)
# where raw wrf outputs will be copied on scratch
raw_scratch_dir = scratch_dir.joinpath("raw")
raw_scratch_dir.mkdir(exist_ok=True)
# where initially restacked data will be stored on scratch_space
restack_scratch_dir = scratch_dir.joinpath("restacked")
restack_scratch_dir.mkdir(exist_ok=True)
slurm_dir = base_dir.joinpath("slurm")
slurm_dir.mkdir(exist_ok=True)

# this env var is always defined if notebook started with anaconda-project run
project_dir = Path(os.getenv("PROJECT_DIR"))
ap_env = project_dir.joinpath("envs/default")
# cp_script = project_dir.joinpath("restack_20km/mp_cp.py") not used on Chinook, $ARCHIVE not accessible from compute nodes
restack_script = project_dir.joinpath("restack_20km/restack.py")
forecast_times_script = project_dir.joinpath("restack_20km/forecast_times.py")
luts_fp = project_dir.joinpath("restack_20km/luts.py")

## 1 - Restack data

This is the main lift of the pipeline and it applies to a single WRF group (again, "group" meaning a specific model / scenario combination) for any variables and years specified. It restacks the WRF outputs, which means extracting the data for all variables in a single hourly WRF file and combining them into new files grouped by variable and year. It then assigns useful metadata and restructures the files to achieve greater usability (note - this was previously a separate step, but the storage of essentially duplicate intermediate data was not efficient).

As mentioned above, this pipeline is currently configured to run for all potential combinations of variables / years for each group. This section will demonstrate execution of all the processing steps required to restack one single WRF group, NCAR-CCSM4 historical, and then will proceed to string them all together for processing the remaining WRF groups.

### 1.1 - Copy WRF data to scratch space

If not present on the filesystem (as is the case at the time of developing the current code) then the WRF data need to be copied over. This is done from tape storage if working on Chinook.

This step will copy the annual subdirectory(ies) containing the WRF outputs for all specified years to scratch space for efficient reading. Given the location of the source data on $ARCHIVE, which requires files to be brought back online from tape to read them, this step is very time consuming.

There are two steps required here:
1. "stage" the files that are on tape storage - i.e., read them from tape to a temporary spot
2. copy the files to scratch space for persistence

It takes a very long to to `batch_stage` an entire group directory. This does not make for a useful experience when trying to execute this pipeline. Given that this transfer needs to be initiated from a login node*, it will be useful to have commands that 
1) determine what files (years) are missing from the scratch space
2) checks to see what of those necessary files (years) are currently "offline" (i.e., in tape storage only and not available for immediate copy)
3) copies files (years) that are online 

\* $ARCHIVE is only visible to the login nodes, so this task cannot be split into subtasks for the compute nodes.

#### 1.1.1 - Specify the WRF group and years to be processed

Set the parameters for processing. The `wrf_dir` specifies the path to the WRF group.

In [3]:
# job parameters
wrf_dir = Path("/archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly")
group = "ccsm_hist"
# years = [2004, 2005]
# to specify all years:
years = luts.groups[group]["years"]

#### 1.1.2 - Stage the files to be processed

Staging all files for a particular WRF group can be done with a single command on the above `wrf_dir`, e.g.:

```
batch_stage -r /archive/DYNDOWN/DIONE/pbieniek/ccsm/hist/hourly
```

However, the cell below will execute the staging for all supplied years. Once this command finishes, then proceed to copying the files. This can take a while.

In [6]:
from subprocess import check_output


stdout = []
for year in years:
    stage_dir = wrf_dir.joinpath(str(year))
    out = check_output(["batch_stage", "-r", stage_dir])
    stdout.append(out)

**Note** - This step should always be done if the data are on `$ARCHIVE` on Chinook, as it should be much more efficient that running the copy without staged (> 3 hours to do a year without staging, ~45 minutes when staged (using 20 cores))

Check that all requested files actually staged:

In [5]:
%time unstaged_fps = main.check_staged(wrf_dir, years)

Requested years: [1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983
 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997
 1998 1999 2000 2001 2002 2003 2004 2005]
All files are staged
CPU times: user 86 ms, sys: 102 ms, total: 188 ms
Wall time: 2.42 s


[]

Ensure yearly subdirectories are present before starting the copying:

In [5]:
main.make_yearly_scratch_dirs(group, years, raw_scratch_dir)

#### 1.1.3 - Copy staged files to `scratch_dir`

Iterate over years and copy the files in parallel with `multiprocesing.Pool`:

In [12]:
ncpus = 20
clobber = "all"


group_dir = raw_scratch_dir.joinpath(group)
for year in tqdm(years, total=len(years), desc=f"Copying files for {len(years)} years"):
    src_dir = wrf_dir.joinpath(str(year))
    dst_dir = group_dir.joinpath(str(year))
    # set third arg to False for no-clobber
    args = [(fp, dst_dir.joinpath(fp.name), clobber) for fp in src_dir.glob("*.nc")]
    
    with Pool(ncpus) as pool:
        out = [out for out in tqdm(pool.imap(main.sys_copy, args), total=len(args), desc=f"Year: {year}")]

Copying files for 20 years:   0%|          | 0/20 [00:00<?, ?it/s]

Year: 1984:   0%|          | 0/8760 [00:00<?, ?it/s]

Year: 1985:   0%|          | 0/8760 [00:00<?, ?it/s]

Year: 1986:   0%|          | 0/8760 [00:00<?, ?it/s]

Year: 1987:   0%|          | 0/8760 [00:00<?, ?it/s]

...

Year: 1999:   0%|          | 0/8760 [00:00<?, ?it/s]

Year: 2000:   0%|          | 0/8760 [00:00<?, ?it/s]

Year: 2001:   0%|          | 0/8760 [00:00<?, ?it/s]

Year: 2002:   0%|          | 0/8760 [00:00<?, ?it/s]

Year: 2003:   0%|          | 0/8760 [00:00<?, ?it/s]

##### Check progress of copying to scratch

Run the cell below to check the progress of the copy for the current arguments (this can take a while):

In [4]:
# modify this to show what entire years are present in scratch_dir

wrf_fps, existing_scratch_fps = main.check_raw_scratch(wrf_dir, group, years, raw_scratch_dir)

All years present on scratch space


This cell below can be used as a quick check to identify any files that didn't copy properly:

In [7]:
flag_fps = []

for year in years:
    year_scratch_dir = raw_scratch_dir.joinpath(group, year)
    flag_fps.extend(check_scratch_file_sizes(year_scratch_dir, ncpus=20))

Then, re-copy any missing files derived from that check:

In [ ]:
main.recopy_raw_scratch_files(flag_fps, wrf_dir)

**Note** - if there is a large number of missing files, it might be more efficient to use the intial section above for copying in batch.

### 1.2 - Restack the data

Now that the WRF outputs are available on the scratch filesystem for faster access, execute the restacking script on all variables of interest.

#### 1.2.1 - Make forecast time table

Tables of forecast time values and filenames are used for interpolating the "accumulation" variables, such as snow and precipitation. This should be done after an entire year's worth of data has successfully copied from `$ARCHIVE` to scratch space, because this table will be referenced for the filepaths and timestamp information to restack. Plus this step utilizes the compute nodes which cannot see `$ARCHIVE`.

**Note** - it is currently unknown why there is an "accumulation fix" needed at all. There could be some info on this lurking somewhere.

Create the slurm script for getting the forecast times:

In [17]:
ncpus = 24
partition = "t1small"

In [32]:
# since this is only done once for a group with all files, only need to specify group (no year(s))
sbatch_fp = slurm_dir.joinpath(f"get_forecast_times_{group}.slurm")
sbatch_out_fp = slurm_dir.joinpath(f"get_forecast_times_{group}_%j.out")
wrf_scratch_dir = raw_scratch_dir.joinpath(group)
sbatch_head = main.make_sbatch_head(slurm_email, partition, conda_init_script, ap_env)
main.write_sbatch_forecast_times(sbatch_fp, sbatch_out_fp, wrf_scratch_dir, anc_dir, forecast_times_script, ncpus, sbatch_head)

Forecast times slurm commands written to /center1/DYNDOWN/kmredilla/wrf_data/slurm/get_forecast_times_ccsm_hist.slurm


Submit the script:

In [34]:
# takes > 30 minutes to run on Chinook
job_id = main.submit_sbatch(sbatch_fp)

#### 1.2.2 - Ensure ancillary WRF geogrid file is present

The restacking will rely on a WRF geogrid data file for determining correct spatial projection information, and for correctly rotating data for wind variables. Make sure that it is present in the `anc_dir` directory:

In [2]:
import shutil


if not geogrid_fp.exists():
    # the original location of this file is not known, in case it is ever deleted
    #  from this source location it might still be available on Poseidon at 
    #  /workspace/Shared/Tech_Projects/wrf_data/project_data/ancillary_wrf_constants/geo_em.d01.nc
    shutil.copy("/import/SNAP/wrf_data/project_data/ancillary_wrf_constants/geo_em.d01.nc", geogrid_fp)

#### 1.2.3 - Run the restacking with slurm

Make the slurm scripts for restacking data for a particular variable and year.

In [18]:
varnames = luts.varnames
ncpus = 24
partition = "t1small"
years = luts.groups[group]["years"]

In [71]:
sbatch_fps = []
year_str = f"{years[0]}-{years[-1]}"
for varname in varnames:
    # write to .slurm script
    sbatch_fp = slurm_dir.joinpath(f"restack_{group}_{year_str}_{varname}.slurm")
    # filepath for slurm stdout
    sbatch_out_fp = slurm_dir.joinpath(f"restack_{group}_{year_str}_{varname}_%j.out")
    sbatch_head = main.make_sbatch_head(
        slurm_email, partition, conda_init_script, ap_env
    )

    args = {
        "sbatch_fp": sbatch_fp,
        "sbatch_out_fp": sbatch_out_fp,
        "restack_script": restack_script,
        "luts_fp": luts_fp,
        "geogrid_fp": geogrid_fp,
        "anc_dir": anc_dir,
        "restacked_dir": restack_scratch_dir,
        "group": group,
        "fn_str": luts.groups[group]["fn_str"],
        "years": years,
        "varname": varname,
        "ncpus": ncpus,
        "sbatch_head": sbatch_head,
    }

    main.write_sbatch_restack(**args)
    sbatch_fps.append(sbatch_fp)

Remove existing slurm output scripts if you fancy it:

In [72]:
for varname in varnames:
    _ = [fp.unlink() for fp in list(slurm_dir.glob(f"*{group}_{year_str}_{varname}_*.out"))]

Submit the `.slurm` scripts with `sbatch`:

In [73]:
job_ids = [main.submit_sbatch(fp) for fp in sbatch_fps]

## 2 - Quality Check

Here we will use the `restack_20km.validate_restacked_file` function to do ensure that ALL* files:

* have the correct data, by checking a random time slice against the original WRF output file (done inside function)
* have the correct metadata
* ensure that all files open correctly, which is done via completion of the first two bullet points

*accumulation and wind variables will not be expected to match raw data!

### 2.1 - Ensure that the restacked files have expected data
...for all relevant variables.

Get all of the new file paths, and feed to run `validate_restacked_file`:

In [4]:
group_fn_str = luts.groups[group]["fn_str"]
all_wrf_fps = list(restack_scratch_dir.glob(f"*/*{group_fn_str}*.nc"))
args = [(fp, raw_scratch_dir) for fp in all_wrf_fps]

In [5]:
tic = time.perf_counter()

with Pool(20) as pool:
    new_rows = [
        result for result in tqdm.tqdm(
            pool.imap_unordered(main.validate_restacked_file, args), total=len(args))
    ]

print(f"Time elapsed: {round((time.perf_counter() - tic) / 60)}m")

100%|█████████████████████████████████████████████████████████████████████████████████████████| 1656/1656 [09:56<00:00,  2.78it/s]

Time elapsed: 10m


Put results into a dataframe:

In [6]:
results_df = pd.DataFrame(new_rows)
results_df

,model,scenario,variable,timestamp,match,meta
0,NCAR-CCSM4,historical,snow,1987-09-11_01,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
1,NCAR-CCSM4,historical,snow,1993-09-11_01,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
2,NCAR-CCSM4,historical,snow,2000-09-11_01,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
3,NCAR-CCSM4,historical,snow,1988-09-11_01,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
4,NCAR-CCSM4,historical,snow,1973-09-11_01,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
...,...,...,...,...,...,...
1651,NCAR-CCSM4,historical,v,None,None,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
1652,NCAR-CCSM4,historical,slp,1984-11-21_17,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
1653,NCAR-CCSM4,historical,slp,1973-02-23_04,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."
1654,NCAR-CCSM4,historical,slp,2000-02-23_04,True,"{'time': {'time zone': 'UTC'}, 'lat': {'standa..."


Where there any mismatches between slices from restacked data and expected raw values?

In [9]:
any(results_df["match"] == False)

False

Good.

### 2.2 - Ensure that restacked files have correct metadata

Have a look at the metadata from the first file:

In [10]:
results_df.iloc[0]["meta"]

{'time': {'time zone': 'UTC'},
 'lat': {'standard_name': 'latitude',
  'title': 'Latitude',
  'units': 'degrees_north',
  'valid_max': 90.0,
  'valid_min': -90.0},
 'lon': {'standard_name': 'longitude',
  'title': 'Longitude',
  'units': 'degrees_east',
  'valid_max': 180.0,
  'valid_min': -180.0},
 'xc': {'standard_name': 'projection_x_coordinate', 'units': 'm'},
 'xc_shape': (262,),
 'yc': {'standard_name': 'projection_y_coordinate', 'units': 'm'},
 'yc_shape': (262,),
 'crs': {'crs_wkt': 'PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",ELLIPSOID["unknown",6370000,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["unknown",METHOD["Polar Stereographic (variant B)",ID["EPSG",9829]],PARAMETER["Latitude of standard parallel",64,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8832]],PARAMETER["Longitude of origin",-152,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8833]],PARAMETER["False easting",

And verify that expected metadata from all files is exactly the same:

In [14]:
np.all([row[1]["meta"] == results_df.iloc[0]["meta"] for row in results_df.iterrows()])

True